1. 문서의 내용을 읽는다
2. 문서를 쪼갠다
    - 토큰 수 초과로 답변을 생성하지 못할 수도 있고
    - 문서가 길면 (인풋이 길면) 답변 생성이 오래 걸림
3. 문서 임베딩 후 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [52]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

spltr=RecursiveCharacterTextSplitter(
    chunk_size=1500, # 문서를 쪼갤 때 하나의 청크가 가질 수 있는 토큰 수
    chunk_overlap=200, # 문서를 자를 때 어느정도 겹치게 할 지 (앞뒤 문맥을 같이 전달해주기 위함)
)

loader=Docx2txtLoader('./tax.docx')
doc_list=loader.load_and_split(text_splitter=spltr)

In [53]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv() # 임베딩 과정에도 환경변수 필요하므로 불러와줘야함

embedding=OpenAIEmbeddings(model='text-embedding-3-large')

In [ ]:
from langchain_chroma import Chroma

# 새로 만들기
db=Chroma.from_documents(
    documents=doc_list,
    embedding=embedding, 
    collection_name='chroma-tax',
    persist_directory='./chroma'
)

# 이미 만든 경우 가져다 쓰기
db=Chroma(collection_name='chroma-tax', persist_directory='./chroma', embedding_function=embedding)


In [55]:
query='연봉 5000만원인 직장인의 소득세는 얼마인가요?'
retrieved_docs=db.similarity_search(query, k=3)

In [56]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model='gpt-4o')

In [57]:
# prompt=f"""[Identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [Context]를 참고해서 사용자의 질문에 답변해주세요

# [Context]
# {retrieved_docs}

# Question: {query}
# """

In [58]:
# ai_msg=llm.invoke(prompt)
# ai_msg.content

In [59]:
from langchain import hub

prompt=hub.pull("rlm/rag-prompt")

In [60]:
from langchain.chains import RetrievalQA

qa_chain=RetrievalQA.from_chain_type(
    llm,
    retriever=db.as_retriever(),
    chain_type_kwargs={"prompt": prompt}
    
)

In [61]:
ai_msg=qa_chain.invoke({"query":query})
ai_msg

{'query': '연봉 5000만원인 직장인의 소득세는 얼마인가요?',
 'result': '그 질문에 대한 정확한 소득세 금액을 계산하기 위해서는 해당 직장인의 소득세율, 공제 항목, 그리고 기타 관련 세금 정보를 알아야 합니다. 일반적으로 한국의 소득세는 누진세 구조로 되어 있어, 연봉 5,000만원인 경우 소득세는 세율과 공제에 따라 다를 수 있습니다. 정확한 금액을 알고 싶다면 국세청의 소득세 계산기를 사용하거나 세무 전문가와 상담하는 것이 좋습니다.'}